In [1]:
!pip install -r requirements.txt

  Attempting uninstall: openai
    Found existing installation: openai 1.56.0
    Uninstalling openai-1.56.0:
      Successfully uninstalled openai-1.56.0


In [23]:
import json
import sys
import os
from openai import OpenAI
sys.path.append("src")

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
%reload_ext autoreload

In [29]:
from wiki_extract import Extracter
from wiki_gen import WikiGen

In [4]:
page = Extracter('Присоединение Крыма к Российской Федерации')

In [5]:
page.get_references()

In [6]:
page.links

{'cite_note-1': [('https://tass.ru/info/6222164',
   'История воссоединения Крыма с Россией  - ТАСС')],
 'cite_note-2': [('https://www.interfax.ru/russia/365492',
   'Крым вошел в состав Российской Федерации')],
 'cite_note-pawn-3': [('https://web.archive.org/web/20160817161027/https://www.stratfor.com/analysis/crimea-russias-little-pawn',
   'Архивная копия')],
 'cite_note-4': [('https://www.telegraph.co.uk/news/worldnews/europe/ukraine/10666557/Keep-out-of-Ukraine-America-tells-Russia.html',
   'Keep out of Ukraine, America tells Russia'),
  ('https://web.archive.org/web/20180619042544/https://www.telegraph.co.uk/news/worldnews/europe/ukraine/10666557/Keep-out-of-Ukraine-America-tells-Russia.html',
   'Архивировано')],
 'cite_note-5': [('https://cyberleninka.ru/article/n/istoriko-pravovye-osnovy-vhozhdeniya-kryma-v-sostav-rossiyskoy-federatsii',
   'Историко-правовые основы вхождения Крыма в состав Российской Федерации')],
 'cite_note-6': [('https://news.ru/russia/10-voprosov-o-priso

In [7]:
page.get_outline()

Creating outline: 100%|██████████| 85/85 [00:00<00:00, 13669.56it/s]


In [8]:
page.get_reference_positions()

Calculating reference positions: 100%|██████████| 25/25 [00:00<00:00, 24155.17it/s]


{'cite_note-1': (('h1', 'Присоединение Крыма к Российской Федерации'), 1),
 'cite_note-2': (('h1', 'Присоединение Крыма к Российской Федерации'), 1),
 'cite_note-pawn-3': (('h2', 'Решающие события'), 1),
 'cite_note-4': (('h2', 'Решающие события'), 1),
 'cite_note-5': (('h2', 'Решающие события'), 3),
 'cite_note-6': (('h2', 'Решающие события'), 3),
 'cite_note-7': (('h2', 'Решающие события'), 5),
 'cite_note-8': (('h2', 'Предыстория'), 1),
 'cite_note-9': (('h2', 'Предыстория'), 1),
 'cite_note-10': (('h3', 'Крым в составе Украины'), 1),
 'cite_note-polunov-11': (('h3', 'Крым в составе Украины'), 4),
 'cite_note-12': (('h3', 'Крым в составе Украины'), 1),
 'cite_note-13': (('h3', 'Референдум 16 марта 2014\xa0года'), 2),
 'cite_note-14': (('h3', 'Международные отношения'), 4),
 'cite_note-16': (('h3', 'Международные отношения'), 4),
 'cite_note-17': (('h3', 'Международные отношения'), 4),
 'cite_note-21': (('h3', 'Международные отношения'), 4),
 'cite_note-22': (('h3', 'Вложения в эконо

In [9]:
print(page.get_text())

Присоединение Крыма к Российской Федерации
Присоединение Крыма к Российской Федерации — вхождение в состав Российской Федерации на основе референдума в марте 2014 года Крымского полуострова в пределах административных границ регионов Украины — Автономной Республики Крым и города Севастополя. На вошедших в состав России территориях Крыма были созданы новые субъекты Российской Федерации — Республика Крым и город федерального значения Севастополь.
Решающие события
Референдуму о вхождении Крыма в состав России предшествовали длительные общественно-политические акции на Украине ( Евромайдан ). 18-20 февраля 2014 года в Киеве произошли кровавые столкновения между сторонниками Евромайдана и украинскими силовиками, что привело к гибели десятков людей и смещению Виктора Януковича с поста президента Украины, существенным переменам в государственной власти, что значительной частью населения, в том числе в Крыму и на Донбассе, было воспринято как государственный переворот.

Недовольство русских об

In [10]:
page.outline

{('h1',
  'Присоединение Крыма к Российской Федерации'): 'Присоединение Крыма к Российской Федерации — вхождение в состав Российской Федерации на основе референдума в марте 2014\xa0года Крымского полуострова в пределах административных границ регионов Украины — Автономной Республики Крым и города Севастополя [1]. На вошедших в состав России территориях Крыма были созданы новые субъекты Российской Федерации\xa0— Республика Крым и город федерального значения Севастополь [2].',
 ('h2',
  'Решающие события'): 'Референдуму о вхождении Крыма в состав России предшествовали длительные общественно-политические акции на Украине ( Евромайдан ). 18-20 февраля 2014\xa0года в Киеве произошли кровавые столкновения между сторонниками Евромайдана и украинскими силовиками, что привело к гибели десятков людей и смещению Виктора Януковича с поста президента Украины, существенным переменам в государственной власти, что значительной частью населения, в том числе в Крыму и на Донбассе, было воспринято как го

In [ ]:
page.newspaper_ref()

  8%|▊         | 3/36 [00:14<03:15,  5.92s/it]

In [14]:
def save_html(html_code, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(html_code)


def save_citations(citations, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(citations, f, ensure_ascii=False, indent=4)

In [15]:
def get_article(name):
    print('Article name: ', name)
    page = Extracter(name)
    page.get_references()
    page.get_outline()
    page.get_reference_positions()
    print('Text length: ', len(page.get_text()))
    print('Number of links: ', len(page.links))
    save_html(page.response.text, "Статьи/Html/" + name + '.html')
    save_citations(page.links, "Статьи/Ссылки/" + name + '.json')

In [3]:
article_names = [
    'Python',
    'Летние Олимпийские игры 2024',
    'Квантовый компьютер',
    'Присоединение Крыма к Российской Федерации',
    'Сколково (инновационный центр)',
    'Tomb Raider (игра, 2013)',
    'Чёрная дыра',
    'Экономика США',
    'Искусственный интеллект',
    'COVID-19'
]

In [ ]:
def save_texts(article_name, texts, number):

    directory = os.path.join("Статьи", "Источники", article_name)

    os.makedirs(directory, exist_ok=True)

    for i, text in enumerate(texts, start=number):
        file_path = os.path.join(directory, f"source_{i}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text)

In [ ]:
num = 1
for _, texts in page.ref_texts.items():
    save_texts(page.name, texts, num)
    num += len(texts)

In [30]:
oclient = OpenAI(api_key=KEY, base_url= URL)

wiki_writer = WikiGen(oclient)

for name in article_names:

    json_str = wiki_writer.get_subqueries(name)

    if json_str:
        data = json.loads(json_str)
        directory = os.path.join("Генерация", "Подзапросы", name)
        os.makedirs(directory, exist_ok=True)
        filename = name.replace(" ", "_") + '.json'
        file_path = os.path.join(directory, filename)
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)